# Massive Machine Learning Pipelines with scikit-learn - Part 1

The objective is to build a scikit-learn pipeline that takes in raw data, transforms many different subsets of data, learns from it, tunes hyperparameters and is then able to make predictions. The final outcome will be a single sckit-learn estimator that does it all. This estimator will then be able to be used in the future to make predictions in a single step.

This tutorial includes information on how to:
* Categorize columns into distinct groups
* Apply independent transformations to each column grouping
* Create a single pipline that handles all the steps of transforming, modeling, validating, and predicting
* Build custom transformers to create new features
* Save our final model to disk to be used to make future predictions

## Focus on transformations
Although this builds a complete machine learning pipeline, most of it will focus on how to transform and prepare the data for the machine learning models.

## Assume fundamentals of scikit-learn
This tutorial assumes you are familiar with doing machine learning with scikit-learn. At a minimum, you need to know what a scikit-learn estimator is and how it behaves

## Prior Issues with transformations in scikit-learn
Up until the release of scikit-learn version 0.20 in September, 2018, there was no easy way to apply separate transformations to different subsets of the data. Additionally burdensome, was encoding of categorical features. For instance, building a single pipeline to handle input data that contained a mix of continuous and categorical variables was not trivial and a huge issue. Many divergent workflows were built to accommodate such transformations.

The addition of the `ColumnTransformer` and upgrade to the `OneHotEncoder` in version 0.20 alleviated these painful issues. Building an entire pipeline in scikit-learn was not pleasant before these additions. A previous post of mine details the exciting new workflow that became possible.

## Real-World Data
In this post, we will work with the [Ames, Iowa housing dataset][0] from a popular Kaggle competition. You are given a dataset with 79 features with the objective of learning a model to predict the sale price.

[0]: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

## Begin by reading in data
Let's begin by reading our training data into a pandas DataFrame.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 100)
housing = pd.read_csv('data/train.csv')
housing.head(3)

Output the number of rows and columns.

In [ ]:
housing.shape

### Remove the sale price
The target variable to predict is the sale price. We will remove it and assign to the variable `y` with the `pop` method, which modifies the DataFrame in place.

In [ ]:
# Running this cell twice will cause an error since the columns are no longer in the DataFrame
y = housing.pop('SalePrice').values
y[:5]

## Use the data dictionary to gain a deeper understanding of the problem

A very useful data dictionary is provided that gives descriptions on each column in the dataset. We will rely on it to understand the meaning of the columns and help with making more logical transformations.

Its contents may be printed out directly into the notebook for easy access. For now, look at the first column. It's composed of numeric values, but they are just codes for a type of house.

In [ ]:
print(open('data/data_description.txt').read())

## Classifying columns into groups
One of the most important tasks during this tutorial is going to be the classification of each column into a particular group. Each group will undergo its own set of transformations before being used as input in the machine learning model. A commonly taught approach is to classify each column as either **categorical** or **continuous**. Values in categorical columns are discrete and typically the total number of categories is known. Continuous columns are always numeric and not limited to a known set of values.

### Different transformations for each group
Categorical columns are processed differently than continous ones with each group needing its own set of transformations. For instance, we may want to fill in missing values with the most frequent for categorical, but the mean for continuous. Categorical columns are often one-hot encoded, while continuous columns are often standardized. As we will see, there are different groups of columns than just categorical or continuous and each column group will have its own set of transformations.

## A Mini Machine Learning Pipeline
Before we embark on our massive machine learning pipeline, we'll create a much simpler one that consists of a few columns that are either categorical or continuous. Creating this small pipeline will help understand the larger one.

### Select columns for each group
We'll begin by assigning a few categorical and continuous columns as lists. The columns here are chosen arbirtrarily as the focus is going to be on applying the transformations and building the pipeline.

In [ ]:
cat_cols = ['Neighborhood', 'LotShape', 'OverallQual', 'MasVnrType']
cont_cols = ['GrLivArea', 'GarageArea', 'LotFrontage']

### Build a pipeline for each column group
Each group will go through two transformations. For the categorical columns, the missing values will be filled with the most frequent and then one-hot encoded. For the continuous columns, the missing values will be filled with the mean and then standardized. A scikit-learn `Pipeline` can be used whenever there are two or more transformations that are needed to be applied in succession.

Before we biuld the pipeline, we will import each of the transformers and instantiate them. Notice that the `OneHotEncoder` is constructed with the `handle_unknown` parameter set to 'ignore'. This will help us when predicting new data that have categories not seen in the training set.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

si_cat = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

si_cont = SimpleImputer(strategy='mean')
ss = StandardScaler()

We can now import the scikit-learn `Pipeline` to create the actual pipelines for each column group. To instantiate a `Pipeline`, create a list of two-item tuples where each tuple consists of a **name** and the **transformer**. The name is an arbitrary string that may be used to reference the transformer at a later time. Below, we create two pipelines, `cat_pipe` and `cont_pipe`.

In [ ]:
from sklearn.pipeline import Pipeline

cat_steps = [
    ('si', si_cat),
    ('ohe', ohe)
]
cat_pipe = Pipeline(cat_steps)

cont_steps = [
    ('si', si_cont),
    ('ss', ss)
]
cont_pipe = Pipeline(cont_steps)

By default, scikit-learn will apply the transformations in a pipeline to all of the columns. But, we are only interested in passing the categorical columns through the categorical pipeline and the continuous columns through the continuous pipeline. The excellent `ColumnTransformer` allows us to do this. We instantiate it with a list of three-item tuples where each tuple consists of a **name** of the transformation, the **transformer**, and a list of **columns** to apply the transformation to. In our case, the transformer is a pipeline of individual transformers.

In [ ]:
from sklearn.compose import ColumnTransformer

transformers = [
    ('cat_cols', cat_pipe, cat_cols),
    ('cont_cols', cont_pipe, cont_cols)
]
ct = ColumnTransformer(transformers)

### Visualizing the ColumnTransformer
At this point, we have constructed the machinery to do the transformations. We haven't actually done any transformations, but we are ready to do so. The following shows how our data would flow at this point. Our raw data would be passed to the `ColumnTransformer` which will send the categorical column to the categorical pipeline and the continuous columns to the continuous pipelines. Each pipeline will apply two successive transformations to the data. After each pipeline has completed, the `ColumnTransformer` concatenates the data back together to form a singe transformed dataset.

![](images/simple_columntransformer.png)

### Passing the data through the `ColumnTransformer`

Let's pass our data through the `ColumnTransformer` to obtain our final transformed dataset. Only the columns that appear in either the `cat_cols` or `cont_cols` lists will be transformed. Any other columns will be dropped. Pass in the pandas DataFrame to the `fit_transform` method and you will be returned a numpy array of the transformed data.

In [ ]:
X_t = ct.fit_transform(housing)
type(X_t)

Let's see the shape of the new dataset.

In [ ]:
X_t.shape

We transformed 7 columns and were returned 46. This is entirely due to one-hot encoding. All the other transformers mapped each input column to exactly one output column.

### Create one more pipeline to do machine learning
Our data is now ready to be fed into a machine learning model. We could use the variable `X_t` that was created above, but instead, we will build another pipeline where the first step passes the data through the `ColumnTransformer` and the second passes the transformed data to the machine learning model. We'll use Ridge Regression for learning. Let's create this final two-step pipeline.

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()

steps = [
    ('ct', ct),
    ('ridge', ridge)
]

final_pipe = Pipeline(steps)

### Visualizing the final pipeline

Our final pipeline is a bit complex. It consists of only two steps, but the ColumnTransformer in the first step contains two separate pipelines and each of those pipelines contains two transformations. The image below shows how the data is processed.

![](images/final_pipeline.png)

### Train the model
Let's use this pipeline to train a Ridge regression model. We simply pass the original DataFrame to the `fit` method which will run all the transformations and train the model.

In [ ]:
final_pipe.fit(housing, y);

### Making predictions
After training, we can now make predictions on new dataets that have the same column names as the original. Let's read in the test dataset and assign the Id column to its own variable.

In [ ]:
housing_test = pd.read_csv('data/test.csv')
housing_test.head(3)

We now pass in the test set to the pipeline's `predict` method to get the predictions.

In [ ]:
y_pred = final_pipe.predict(housing_test)
y_pred[:5]

## Saving the model to disk for future use
We can preserve our trained pipeline exactly as it is by saving it to disk with help from the `joblib` library `dump` function. See [joblib documentation][0] for more information. Pass it the pipeline and a name for the new file.

[0]: https://joblib.readthedocs.io/en/latest/persistence.html#persistence

In [ ]:
import joblib
joblib.dump(final_pipe, 'models/minipipeline_ridge.joblib')

## Retrieve the saved model
We can retrieve the model from disk with the `load` function. It has preserved every step of the pipeline. We test that the results are the same by testing that the predictions are the same as the original.

In [ ]:
final_pipe_new = joblib.load('models/minipipeline_ridge.joblib')
y_pred_new = final_pipe_new.predict(housing_test)
(y_pred == y_pred_new).all()

## Submit to Kaggle

We now have a prediction for each test observation and can submit them to Kaggle to get scored and ranked against the other competitors.

### Create a csv of Id and SalePrice
We need to submit a csv file of the row Id and our predicted sale price. We do so with the DataFrame constructor to create a two-column DataFrame.

In [ ]:
sub01 = pd.DataFrame({'Id': housing_test['Id'], 'SalePrice': y_pred})
sub01.head(3)

### File naming and directory structure

From here we can export our DataFrame as a csv. I strongly recommend creating a submissions folder within the data folder and within that folder a new folder for each date that you track submissions. Within that folder is where the submission files will be saved.

In [ ]:
sub01.to_csv('data/submissions/20190710/sub01.csv', index=False)

Our directory structure for the data takes the following shape:

![](images/dir.png)

### Make a submission to Kaggle from python
Kaggle has kindly provided a [python library][0] (`pip install kaggle`) to make submissions programmatically. You'll need to read the documentation to learn how to authenticate your account.

[0]: https://github.com/Kaggle/kaggle-api

In [ ]:
import kaggle

We submit our csv by passing the submission function our file location, a message, and the competition name. It's important to give a good descriptive message so that you can remember how that particular submission was created.

In [ ]:
file = 'data/submissions/20190710/sub01.csv'
message = '''
One hot encoded four categorical columns and standardized three continuous
columns. Modeled with ridge regression with alpha=1
'''
competition = 'house-prices-advanced-regression-techniques'
# kaggle.api.competition_submit(file, message, competition)

### Retrieve score
We can retrieve a list of all of our prior scores with the following function call. The most recent submission will be the first item in the list which we print to the screen.

In [ ]:
all_submissions = kaggle.api.competitions_submissions_list(competition)
all_submissions[0]

The competition uses root mean squared error of the logged housing prices. Our score is .18. Unfortunately there is no easy way to get our place on the leaderboard, but you can just navigate [directly to the Kaggle leaderboard][0] instead.

[0]: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/leaderboard

## Pipeline vs ColumnTransformer
It may not be crystal clear what the difference is between the `Pipeline` and `ColumnTransformer`. The simplest distinction is to think of the `Pipeline` as moving all of the data **vertically** in succession from one transformer to the next. The `ColumnTransformer` splits data **horizontally** into multiple subsets. Each of these subsets gets applied a transformation that is independent of all the other subsets of data. All the transformed subsets are then concatenated together to form a single dataset.

In a `Pipeline`, all columns will be passed into each estimator with the result being passed to the next estimator. A `Pipeline` allows for the last step to be a machine learning model whereas the `ColumnTransformer` only allows transformers.

### All steps in one cell
All the above steps for the our mini machine learning pipeline were written in different cells, which probably makes it harder to see the full structure of the program as it would outside of a Jupyter Notebook/Tutorial

In [ ]:
si_cat = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

si_cont = SimpleImputer(strategy='mean')
ss = StandardScaler()

cat_cols = ['Neighborhood', 'LotShape', 'OverallQual', 'MasVnrType']
cont_cols = ['GrLivArea', 'GarageArea', 'LotFrontage']

cat_steps = [('si', si_cat), ('ohe', ohe)]
cat_pipe = Pipeline(cat_steps)

cont_steps = [('si', si_cont), ('ss', ss)]
cont_pipe = Pipeline(cont_steps)

transformers = [('cat_cols', cat_pipe, cat_cols), ('cont_cols', cont_pipe, cont_cols)]
ct = ColumnTransformer(transformers)

steps = [('ct', ct), ('ridge', ridge)]
final_pipe = Pipeline(steps)
final_pipe.fit(housing, y)
y_pred = final_pipe.predict(housing_test)
sub01 = pd.DataFrame({'Id': housing_test['Id'], 'SalePrice': y_pred})
sub01.to_csv('data/submissions/20190710/sub01.csv', index=False)